In [1]:
import re, subprocess, pyodbc , json
import numpy as np # linear algebra 
from sqlalchemy import func, create_engine
from config import mongoConn, connstr  
import pandas as pd
from faker import Faker
fake = Faker()

In [2]:
engine = create_engine(connstr)
conn = engine.connect()
query = """ \
SELECT QPK, QGROUP, IdText, QuestionTypeCode, QuestionTypeID, QTEXT    
FROM  vw_MetricsCompositeKeys
WHERE PK_Form LIKE '%2022-Q1-CIO%'
"""  
df = pd.read_sql(query,con=conn) 
conn.close()


,QPK,QGROUP,IdText,QuestionTypeCode,QuestionTypeID,QTEXT
0,27000,3201,,SDESC,64,The goal of the Identify metrics section is to...
1,27001,3201,1.1,CIO2,39,"For each FIPS 199 impact level, what is the nu..."
2,27002,3201,1.1.1,LABEL,18,Organization-Operated Systems
3,27003,3201,1.1.2,LABEL,18,Contractor-Operated Systems
4,27004,3201,1.1.3,LABEL,18,Systems (from 1.1.1 and 1.1.2) with Security ATO
...,...,...,...,...,...,...
189,27437,3216,10.6.4.1,YN,1,Does the agencys communications strategy inclu...
190,27438,3216,10.6.5,YN,1,Has the agency determined a process for sharin...
191,27439,3216,,LABEL,18,[1]The elapsed time from when the report is su...
192,27440,3216,,LABEL,18,[2]The elapsed time from when a report is subm...


In [ ]:
engine = create_engine(connstr)
conn = engine.connect()
query = """ \
SELECT QPK, QGROUP, IdText, QuestionTypeCode , QuestionTypeCode, QTEXT, help_text   
FROM  vw_MetricsCompositeKeys
WHERE PK_Form LIKE '%2022-Q1-CIO%'
"""  
df = pd.read_sql(query,con=conn) 
conn.close()

In [76]:
offset = 10
qg = 3200
fp = 3000

In [77]:
sections = [ 
{'id':'7a', 'text': 'Ground Truth Testing- Testing Activities'},
{'id':'7b', 'text': 'Ground Truth Testing- Penetration Testing'},
{'id':'7c', 'text': 'Ground Truth Testing- Vulnerability Disclosure Tools'},
{'id':'7d', 'text': 'Ground Truth Testing- Team Exercises'},
{'id':'8', 'text': 'Smart Patching'},
{'id':'9', 'text': 'Vulnerability Disclosure'},
{'id':'10', 'text': 'Resilience'}  
]
df_s = pd.DataFrame(sections)
df_s

,id,text
0,7a,Ground Truth Testing- Testing Activities
1,7b,Ground Truth Testing- Penetration Testing
2,7c,Ground Truth Testing- Vulnerability Disclosure...
3,7d,Ground Truth Testing- Team Exercises
4,8,Smart Patching
5,9,Vulnerability Disclosure
6,10,Resilience


In [78]:
for i,r in df_s.iterrows():
    n = offset+i
    p = f"(@PK_FormPage+{n},@PK_Form, '{r.id}: {r.text}', 'FismaForms/2022/2022_Q1_CIO_{r['id']}.aspx', {n}), ";
    print( p ) 

(@PK_FormPage+10,@PK_Form, '7a: Ground Truth Testing- Testing Activities', 'FismaForms/2022/2022_Q1_CIO_7a.aspx', 10), 
(@PK_FormPage+11,@PK_Form, '7b: Ground Truth Testing- Penetration Testing', 'FismaForms/2022/2022_Q1_CIO_7b.aspx', 11), 
(@PK_FormPage+12,@PK_Form, '7c: Ground Truth Testing- Vulnerability Disclosure Tools', 'FismaForms/2022/2022_Q1_CIO_7c.aspx', 12), 
(@PK_FormPage+13,@PK_Form, '7d: Ground Truth Testing- Team Exercises', 'FismaForms/2022/2022_Q1_CIO_7d.aspx', 13), 
(@PK_FormPage+14,@PK_Form, '8: Smart Patching', 'FismaForms/2022/2022_Q1_CIO_8.aspx', 14), 
(@PK_FormPage+15,@PK_Form, '9: Vulnerability Disclosure', 'FismaForms/2022/2022_Q1_CIO_9.aspx', 15), 
(@PK_FormPage+16,@PK_Form, '10: Resilience', 'FismaForms/2022/2022_Q1_CIO_10.aspx', 16), 


In [79]:
for i,r in df_s.iterrows():
    n = offset+i
    p = f"(@PK_QGroup+{n},'{r.id}','Section {n}: Identify - {r.text}',@PK_Form, @PK_FormPage+{n}, {n}),";
    print( p ) 

(@PK_QGroup+10,'7a','Section 10: Identify - Ground Truth Testing- Testing Activities',@PK_Form, @PK_FormPage+10, 10),
(@PK_QGroup+11,'7b','Section 11: Identify - Ground Truth Testing- Penetration Testing',@PK_Form, @PK_FormPage+11, 11),
(@PK_QGroup+12,'7c','Section 12: Identify - Ground Truth Testing- Vulnerability Disclosure Tools',@PK_Form, @PK_FormPage+12, 12),
(@PK_QGroup+13,'7d','Section 13: Identify - Ground Truth Testing- Team Exercises',@PK_Form, @PK_FormPage+13, 13),
(@PK_QGroup+14,'8','Section 14: Identify - Smart Patching',@PK_Form, @PK_FormPage+14, 14),
(@PK_QGroup+15,'9','Section 15: Identify - Vulnerability Disclosure',@PK_Form, @PK_FormPage+15, 15),
(@PK_QGroup+16,'10','Section 16: Identify - Resilience',@PK_Form, @PK_FormPage+16, 16),


In [80]:
def cleaner(s):
    try:
        s=s.replace('\'','"')
        s=s.replace('�','')
        s=s.replace('nan','')
        s=re.sub('[\u0080-\uFFFF]+', '', s)
    except:
        pass
    return s

In [95]:
l = [] 
s=10
qpk = 27200
for i, r in df_s[:].iterrows():  
    dff = pd.read_csv(fr'C:\Users\Tim\Documents\CIO\{r.id}.csv', dtype=str)
    dff.fillna('') 
    dff = dff.astype(str)
    print( f"\n--{r['id']} {r['text']}")
    print( f"""INSERT INTO fsma_Questions (PK_Question, FormName, FK_QuestionGroup, sortpos, identifier_text, FK_QuestionType, FK_PickList, QuestionText, help_text) VALUES """)
    for ii, rr in dff.iterrows():  
        qtext = cleaner(rr['qtext'])
        qtype = cleaner(rr['qtype'])
        if pd.isna(qtype):
            qtype = '0' 
        qtype = re.sub(r"[^\d]", "", qtype)
        if qtype == '':
            qtype='18'
        qid = re.sub(r"[^\d{1,2}\.]", "", rr['id'])
        if len(qtext) > 10:
            p=f"({qpk}, @FormName, @PK_QGroup+{s},{ii+1},N\'{qid}\', {qtype}, NULL, N\'{qtext}\', NULL)"
            if ii > 0:
                p=f",{p}"
            print(  p  )
            qpk=qpk+1
    s=s+1
    qpk=qpk+25
 
 


--7a Ground Truth Testing- Testing Activities
INSERT INTO fsma_Questions (PK_Question, FormName, FK_QuestionGroup, sortpos, identifier_text, FK_QuestionType, FK_PickList, QuestionText, help_text) VALUES 
(27200, @FormName, @PK_QGroup+10,1,N'', 62, NULL, N'The purpose of this section is to start evaluating how agency testing procedures are currently established, conducted, and performed. Ground truth testing looks to go beyond the assumption that generic vulnerability scanning tools are sufficient for testing system security. Additionally, this section is intended to baseline how well the organization internally communicates the effectiveness of its security testing.', NULL)
,(27201, @FormName, @PK_QGroup+10,2,N'7.1.1', 18, NULL, N' Penetration test (using automated tools only)', NULL)
,(27202, @FormName, @PK_QGroup+10,3,N'7.1.2', 18, NULL, N' Penetration test (using manual, expert, system-specific analysis[2])', NULL)
,(27203, @FormName, @PK_QGroup+10,4,N'7.1.3', 18, NULL, N' Red team